In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,LSTM
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = './hin-eng/hin.txt'

In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1 )]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    target_texts.append(target_text)
    input_texts.append(input_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [4]:
print("Number of samples : ",len(input_texts))
print("Number of unique input tokens : ", num_encoder_tokens)
print("Number of unique output tokens : ", num_decoder_tokens)
print("Max sequence length in input : ", max_encoder_seq_len)
print("Max sequence length in output : ",max_decoder_seq_len)


Number of samples :  2979
Number of unique input tokens :  70
Number of unique output tokens :  90
Max sequence length in input :  107
Max sequence length in output :  123


In [5]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [6]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_len, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype='float32')

In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t-1, target_token_index[char]] = 1
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1
    

In [8]:
encoder_input_data[0].shape

(107, 70)

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [10]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ , _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs,decoder_inputs], encoder_outputs)

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit([encoder_input_data,decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

In [13]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
deocoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i,char) for char in input_token_index.items())
reverse_target_char_index = dict(
    (i,char) for char in target_token_index.items())

def decode_sequence(input_seq):
    state_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1, num_decoder_tokens))
    
    target_seq[0,0,target_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tockens, h, c = decoder_model.predict(
        [target_seq] + state_value)
        
        sample_token_index = np.argmax(output_tockens[0, -1, :])
        smaple_char = reverse_target_char_index[sample_token_index]
        decoded_sentence += smaple_char
        
        if ( smaple_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True
            
        target_seq = np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sample_token_index] = 1
        
        state_value = [h,c]
        
    return decoded_sentence

for seq_index in range(100):
    inuput_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input Sentence : ' , input_texts[seq_index])
    print('Decoded Sentence : ', decoded_sentence)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, None, 70), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "lstm". The following previous layers were accessed without issue: []